## 1. Gemini API 기반 데이터 생성 스크립트
##### ~~pip install -U google-generativeai~~
##### pip install google-genai

In [5]:
from google import genai
from config import config

client = genai.Client(api_key=config.gemini_api_key)

for m in client.models.list():
    print(f"Model Name: {m.name}")
    # print(f"Model Name: {m.name}, Supported Methods: {m.model_fields}")

Model Name: models/embedding-gecko-001
Model Name: models/gemini-2.5-flash
Model Name: models/gemini-2.5-pro
Model Name: models/gemini-2.0-flash-exp
Model Name: models/gemini-2.0-flash
Model Name: models/gemini-2.0-flash-001
Model Name: models/gemini-2.0-flash-exp-image-generation
Model Name: models/gemini-2.0-flash-lite-001
Model Name: models/gemini-2.0-flash-lite
Model Name: models/gemini-2.0-flash-lite-preview-02-05
Model Name: models/gemini-2.0-flash-lite-preview
Model Name: models/gemini-exp-1206
Model Name: models/gemini-2.5-flash-preview-tts
Model Name: models/gemini-2.5-pro-preview-tts
Model Name: models/gemma-3-1b-it
Model Name: models/gemma-3-4b-it
Model Name: models/gemma-3-12b-it
Model Name: models/gemma-3-27b-it
Model Name: models/gemma-3n-e4b-it
Model Name: models/gemma-3n-e2b-it
Model Name: models/gemini-flash-latest
Model Name: models/gemini-flash-lite-latest
Model Name: models/gemini-pro-latest
Model Name: models/gemini-2.5-flash-lite
Model Name: models/gemini-2.5-flas

In [ ]:
from google import genai
from pydantic import BaseModel # 구조화된 출력을 위해 권장
import json
import time
import os

from config import config

# 1. Gemini API 설정
client = genai.Client(api_key=config.gemini_api_key)

# 모델 ID 설정
MODEL_ID = config.gemini_model_id

# 2. 상세 카테고리 정의
categories = [
    "한국사 주요 사건", "기초 물리 법칙", "생활 속 화학 원리", "태양계와 우주 탐사",
    "IT 트렌드 및 컴퓨터 용어", "기초 경제 용어", "세계의 지리와 문화",
    "클래식 음악과 예술가", "스포츠 규칙 및 역사", "기후 변화와 환경 상식"
]

def generate_knowledge_data(category, count=10):
    prompt = f"""
    너는 인공지능 학습을 위한 고품질 한국어 데이터셋 생성 전문가야.
    {category} 분야에 관한 '일반 상식 및 정보' 데이터셋 {count}개를 생성해줘.

    [조건]
    1. 형식은 반드시 JSON 리스트 형태여야 함.
    2. 리스트 내 객체는 "instruction"과 "output" 키만 가질 것.
    3. 질문(instruction)은 간결하고, 답변(output)은 정확한 사실을 바탕으로 1~2문장으로 작성할 것.
    """

    try:
        # 새로운 SDK의 generate_content 방식
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=prompt,
            config={  # config 매개변수 통해 JSON 출력 강제
                'response_mime_type': 'application/json',
            }
        )
        return json.loads(response.text)
    except Exception as e:
        print(f"오류 발생: {e}")
        return None

# 3. 데이터 수집 및 저장
os.makedirs("./output", exist_ok=True)
output_file = "./output/gemini_synthetic_data.jsonl"

for i in range(50):  # 50번 반복하여 총 50 * count개 생성
    target_cat = categories[i % len(categories)]
    print(f"[{i+1}/50] {target_cat} 생성 중...")

    results = generate_knowledge_data(target_cat, count=10)

    if results:
        with open(output_file, "a", encoding="utf-8") as f:
            # 리스트 형태인 경우와 단일 객체인 경우 모두 대응
            if isinstance(results, list):
                for item in results:
                    f.write(json.dumps(item, ensure_ascii=False) + "\n")
            elif isinstance(results, dict):
                # 가끔 모델이 리스트가 아닌 'data': [] 형태의 딕셔너리로 줄 때를 대비
                items = results.get('data', [results])
                for item in items:
                    f.write(json.dumps(item, ensure_ascii=False) + "\n")

    # 무료 티어의 분당 호출 제한(RPM)을 피하기 위해 4~5초간 대기
    time.sleep(4)

print(f"LLM 학습 데이터 생성 완료: {output_file}")

In [1]:
# import google.generativeai as genai  # # Deprecated
# import json
# import time
# import os
#
# from config import config
#
# # 1. Gemini API 설정
# # https://aistudio.google.com/ 에서 무료 API 키를 발급 필요
# genai.configure(api_key=config.gemini_api_key)
# model = genai.GenerativeModel(config.gemini_model_id)
#
# # 2. 상세 카테고리 정의
# categories = [
#     "한국사 주요 사건", "기초 물리 법칙", "생활 속 화학 원리", "태양계와 우주 탐사",
#     "IT 트렌드 및 컴퓨터 용어", "기초 경제 용어", "세계의 지리와 문화",
#     "클래식 음악과 예술가", "스포츠 규칙 및 역사", "기후 변화와 환경 상식"
# ]
#
# def generate_knowledge_data(category, count=10):
#     prompt = f"""
#     너는 인공지능 학습을 위한 고품질 한국어 데이터셋 생성 전문가야.
#     {category} 분야에 관한 '일반 상식 및 정보' 데이터셋 {count}개를 생성해줘.
#
#     [조건]
#     1. 형식은 반드시 JSON 리스트 형태여야 함.
#     2. 리스트 내 객체는 "instruction"과 "output" 키만 가질 것.
#     3. 질문(instruction)은 간결하고, 답변(output)은 정확한 사실을 바탕으로 1~2문장으로 작성할 것.
#     4. 마크다운 기호 없이 순수 JSON 코드만 출력해.
#
#     예시: {{"instruction": "조선왕조실록이란?", "output": "조선 시대 왕들의 재위 기간 기록을 날짜순으로 정리한 역사 기록물입니다."}}
#     """
#
#     try:
#         response = model.generate_content(prompt)
#         # 텍스트에서 JSON 부분만 추출 (가끔 붙는 ```json 제거)
#         clean_text = response.text.replace('```json', '').replace('```', '').strip()
#         return json.loads(clean_text)
#     except Exception as e:
#         print(f"오류 발생: {e}")
#         return None
#
# # 3. 데이터 수집 및 저장
# output_file = "./output/gemini_synthetic_data.jsonl"
#
# for i in range(50):  # 50번 반복하여 총 500개 생성 (무료 티어 분당 호출 제한 주의)
#     target_cat = categories[i % len(categories)]
#     print(f"[{i+1}/50] {target_cat} 생성 중...")
#
#     results = generate_knowledge_data(target_cat, count=10)
#
#     if results:
#         with open(output_file, "a", encoding="utf-8") as f:
#             for item in results:
#                 f.write(json.dumps(item, ensure_ascii=False) + "\n")
#
#     # 무료 티어의 분당 호출 제한(RPM)을 피하기 위해 4~5초간 대기
#     time.sleep(4)
#
# print(f"LLM 학습 데이터 생성 완료: {output_file}")

/opt/anaconda3/envs/data_analysis/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/anaconda3/envs/data_analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/b2/vx12c8bs3bgfsxvbxwr9kddm0000gn/T/ipykernel_23969/1511828891.py:1: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possi

[1/50] 한국사 주요 사건 생성 중...
오류 발생: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
[2/50] 기초 물리 법칙 생성 중...
오류 발생: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.


KeyboardInterrupt: 

## 2. 수천 개 데이터 품질 검수 스크립트
##### Gemini가 간혹 "데이터를 생성해 드릴 수 없습니다"와 같은 거절 메시지를 포함할 수 있음. 이를 필터링하는 검수 코드.

In [ ]:
import pandas as pd

def clean_and_verify(file_path, res_fl_path="./output/final_cleaned_data.jsonl"):
    df = pd.read_json(file_path, lines=True)

    # 1. 중복 질문 제거
    df = df.drop_duplicates(subset=['instruction'])

    # 2. 너무 짧거나 거절 메시지가 포함된 답변 제거
    bad_keywords = ["죄송합니다", "제공할 수 없습니다", "인공지능 비서"]
    df = df[~df['output'].str.contains('|'.join(bad_keywords))]

    # 3. 글자 수 필터링 (너무 짧은 답변 제외)
    df = df[df['output'].str.len() > 5]

    print(f"최종 정제된 데이터 개수: {len(df)}")
    df.to_json(res_fl_path, orient="records", lines=True, force_ascii=False)

clean_and_verify(output_file)

## 3. 학습 데이터 확장 로드맵
##### 수천 개를 넘어 수만 개로 가려면 카테고리를 더 세분화.

##### 세부 카테고리화: "한국사" 대신 "조선 전기 사회상", "구한말 외교 관계" 식으로.

##### 역방향 생성: "A는 B다"라는 문장을 주고 "이 문장이 답이 될 수 있는 질문을 만들어줘"라고 요청하여 데이터의 다양성을 확보.

##### 수집된 데이터 양이 2,000개가 넘어가면, TinyLlama 학습 시 learning_rate를 조금 낮추어(예: 1e-4) 더 정교하게 튜닝.

## 1. 데이터 자동 생성 파이썬 스크립트
##### pip install openai
##### 주제 지정 -> API 호출 -> 결과 검증 -> JSONL 저장

In [ ]:
import openai
import json
import random

from config import config

# OpenAI API 키 설정
client = openai.OpenAI(api_key=config.open_ai_api_key)

# 1. 데이터 생성을 위한 세부 카테고리 정의 (수천 개 확장을 위한 기반)
categories = [
    "한국 역사", "세계사", "기초 과학(물리, 화학, 생물)", "천문학 및 우주",
    "IT 및 컴퓨터 공학", "경제 및 경영 상식", "법률 및 정치", "지리 및 국가",
    "예술 및 클래식 음악", "스포츠 상식", "환경 및 기후 변화"
]

def generate_data(category, count=10):
    """
    GPT를 사용하여 특정 카테고리의 지시사항-답변 쌍을 생성.
    """
    prompt = f"""
    너는 인공지능 학습을 위한 고품질 한국어 데이터셋 생성 전문가야.
    {category} 분야에 관한 '일반 상식 및 정보' 데이터셋 {count}개를 생성해줘.

    [조건]
    1. 형식은 반드시 JSON 리스트 형태여야 함.
    2. 키값은 "instruction"과 "output"으로 구성할 것.
    3. 질문(instruction)은 명확해야 하며, 답변(output)은 1~2문장의 정확한 사실로 작성할 것.
    4. TinyLlama-1.1B 모델이 학습하기 적좋은 짧고 간결한 문체를 사용할 것.

    예시: {{"instruction": "지구에서 가장 높은 산은?", "output": "에베레스트 산입니다."}}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini", # 비용 효율적인 mini 모델 권장
        messages=[{"role": "user", "content": prompt}],
        response_format={ "type": "json_object" } # JSON 출력을 보장
    )

    return response.choices[0].message.content

# 2. 실행 및 파일 저장
output_file = "synthetic_knowledge_data.jsonl"

with open(output_file, "a", encoding="utf-8") as f:
    for i in range(10): # 예시로 10번 반복 (한 번에 10개씩 총 100개)
        category = random.choice(categories)
        print(f"[{i+1}] {category} 데이터 생성 중...")

        try:
            raw_result = generate_data(category, count=10)
            json_result = json.loads(raw_result)

            # JSON 내의 리스트 추출 (GPT 응답 구조에 따라 조정 필요)
            items = json_result.get("data", json_result.get("items", json_result))
            if isinstance(items, dict): # 딕셔너리 형태일 경우 리스트로 변환
                items = list(items.values())[0]

            for item in items:
                f.write(json.dumps(item, ensure_ascii=False) + "\n")
        except Exception as e:
            print(f"에러 발생: {e}")

print(f"데이터 생성이 완료되었습니다. 파일명: {output_file}")

## 2. 수천 개 데이터 구축 시 핵심 전략
##### 시드(Seed) 키워드 활용: 위 스크립트에서 categories 리스트를 아주 상세하게 확장. (예: "한국 역사" -> "조선 시대 왕의 업적", "고구려의 전쟁사" 등) 질문의 깊이가 깊어질수록 데이터의 질이 나아짐.

##### 비용 관리: gpt-4o 보다는 gpt-4o-mini 모델을 추천. 가격이 수십 배 저렴하면서도 일반 상식 데이터 생성 능력은 충분.

### 중복 제거 (Deduplication): 생성된 데이터가 수천 개가 되면 "지구에서 가장 높은 산은?" 같은 질문이 중복될 수 있음. 파이썬의 set이나 Pandas를 사용하여 instruction 기준 중복을 반드시 제거.

## 3. 데이터 검증 및 필터링 (품질 관리)
대량 생성된 데이터 중에는 모델이 잘못된 정보(Hallucination) 생성 가능.

키워드 필터링: 답변에 "죄송합니다", "인공지능으로서" 등의 단어가 들어간 데이터 삭제.

길이 필터링: instruction이 너무 짧거나 output이 지나치게 긴 데이터는 TinyLlama 학습에 방해 되니 제거.

## 4. 학습 전 최종 데이터 통합
스크립트로 만든 synthetic_knowledge_data.jsonl과 기존 수동 작성 데이터 merge

In [ ]:
import pandas as pd

# 두 파일을 읽어서 합치기
df1 = pd.read_json("manual_data.jsonl", lines=True)
df2 = pd.read_json("synthetic_knowledge_data.jsonl", lines=True)

final_df = pd.concat([df1, df2]).drop_duplicates(subset=['instruction'])
final_df.to_json("total_train_data.jsonl", orient="records", lines=True, force_ascii=False)